In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import *
from sklearn.tree import *
from sklearn.metrics import *
from sklearn.linear_model import *
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler, StandardScaler, LabelEncoder, MinMaxScaler
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV, train_test_split
from torch.utils.data import DataLoader, TensorDataset, Dataset
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence, pad_sequence
from tweedejaars_project import *
import torch.nn.functional as F
from datetime import datetime

In [11]:
""" Feature engineering import """

def features(df):
    # forecast
    df['residual_load'] = df['forecast_demand'] - df['forecast_solar'] - df['forecast_wind']
    df['forecast_solar_delta'] = df['forecast_solar'].diff()
    df['forecast_wind_delta'] = df['forecast_wind'].diff()

    for i in range(0, len(df), 15):
        df.loc[i+1:i+14, 'forecast_solar_delta'] = df['forecast_solar_delta'][i]
        df.loc[i+1:i+14, 'forecast_wind_delta'] = df['forecast_wind_delta'][i]

    # diffs
    df['min_price_diff'] = df['min_price_published'].diff() + df['min_price_published']
    df['downward_dispatch_diff'] = df['downward_dispatch_published'].diff() + df['downward_dispatch_published']
    df['igcc_down_diff'] = df['igcc_contribution_down_published'].diff() + df['igcc_contribution_down_published']
    df['dispatch_diff'] = df['upward_dispatch_published'] - df['downward_dispatch_published']
    df['igcc_diff'] = df['igcc_contribution_up_published'] - df['igcc_contribution_down_published']

    # balance
    df['balance'] = df['residual_load']

    # boolean
    df['import_zero'] = df['import_capacity'] == 0

    # external markets
    df['import_capacity_left'] = df['import_capacity'] + df['igcc_contribution_down_published'] - df['igcc_contribution_up_published']

    return df

In [12]:
""" Prepare Dataset """

# load data
df = load_df()

# clean the forecasts by updating with last known
# fix wind diff
# set the last 46 missing values in settlement price realised to zero
# the last 8700 values of vwap (and last 16 of igcc) are missing, therefore trim
# igcc misses 16 values, replace for 0.0
# settlement price best guess replaced with current height setllement price (nan _> 0, abs of min values) or boolean settlement price

# add features and clean features of nan
df = features(df)

# features with at least 0.04 correlation with target
highly_correlated_features = ['naive_strategy_action', 'min_ptu_price_known',
                              'min_price_published', 'forecast_wind',
                              'time_since_last_two_sided',
                              'two_sided_daily_count', 'mid_price_published',
                              'vwap_avg', 'vwap_std', 'vwap_qty_sum',
                              'minute_in_ptu', 'downward_dispatch_published',
                              'settlement_price_bestguess', 'import_capacity',
                              'upward_dispatch_published',
                              'settlement_price_realized', 'forecast_solar',
                              'igcc_contribution_down_published', 'vwap_max',
'min_price_diff',
'residual_load',
'balance',
'import_zero',
'dispatch_diff',
'downward_dispatch_diff',
'import_capacity_left',
'igcc_down_diff',
'forecast_wind_delta'
]

highly_correlated_features = ['naive_strategy_action',
                              'time_since_last_two_sided', 
                              'two_sided_daily_count', 'mid_price_published',
                              'vwap_avg', 'vwap_std', 'vwap_qty_sum',
                              'minute_in_ptu', 'settlement_price_realized']

target = 'target_two_sided_ptu'

In [2]:
# split data into features and target
targets_numpy = df[target]
features_numpy = df[highly_correlated_features]

# train test split
# could be replaced with scikit-learn TimeSeriesSplit
X_train, X_test = np.split(features_numpy, [int(.7 * len(features_numpy))])
y_train, y_test = np.split(targets_numpy, [int(.7 * len(targets_numpy))])

# create feature and targets tensor for train and test set.
featuresTrain = torch.Tensor(X_train.to_numpy(dtype=np.float64))
targetsTrain = torch.Tensor(y_train.to_numpy(dtype=np.float64))

featuresTest = torch.Tensor(X_test.to_numpy(dtype=np.float64))
targetsTest = torch.Tensor(y_test.to_numpy(dtype=np.float64))

# force the right dimension
targetsTrain = targetsTrain.unsqueeze(1)
targetsTest = targetsTest.unsqueeze(1)

# Pytorch train and test sets
train = torch.cat((featuresTrain, targetsTrain), dim=1)
test = torch.cat((featuresTest, targetsTest), dim=1)

print(test.shape)

# batch_size, epoch and iteration
first_batch_size = 15
n_iters = 5000
num_epochs = 1 + int(n_iters / (len(X_train) / first_batch_size))

# data loader makes batches and iterable
train_loader = DataLoader(train, batch_size = first_batch_size, shuffle=False)
test_loader = DataLoader(test, batch_size = first_batch_size, shuffle=False)

/home/tycho/miniconda3/envs/tweedejaars_project/lib/python3.12/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
/home/tycho/miniconda3/envs/tweedejaars_project/lib/python3.12/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'Series.swapaxes' is deprecated and will be removed in a future version. Please use 'Series.transpose' instead.
  return bound(*args, **kwds)


torch.Size([66074, 9])


In [18]:
""" Create RNN Model """

class RNNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(RNNModel, self).__init__()
        
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.output_dim = output_dim
        
        # RNN
        self.rnn = nn.RNN(input_dim, hidden_dim, num_layers, batch_first=True,
                          nonlinearity='relu')
        
        # Collection layer
        self.fc = nn.Linear(hidden_dim, output_dim)

        # Range redistribution [0,1]
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):

        # Initialize hidden state with zeros
        h0 = torch.zeros(self.num_layers, x.shape[0], self.hidden_dim,
                         requires_grad=True)
        
        try:
            h0 = hn
        except:
            pass

        # One time step
        rnn_out, hn = self.rnn(x, h0)
        
        fc_out = self.fc(rnn_out[:, -1])  # last output

        output = self.sigmoid(fc_out)

        return output


# Create RNN
input_dim =  8    # input dimension
hidden_dim = 60   # hidden layer dimension
num_layers = 2    # number of hidden layers
output_dim = 15   # output dimension
seq_length = 15   # the number of time steps in each sequence

model = RNNModel(input_dim, hidden_dim, num_layers, output_dim)

# SGD Optimizer
learning_rate = 0.0005
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=0.1)

In [19]:
""" Run model """

batch_size = int(first_batch_size / seq_length) 
loss_list = []
iteration_list = []
accuracy_list = []
count = 0
error = nn.BCEWithLogitsLoss()
test_iter = iter(test_loader)

splits = get_splits(df, highly_correlated_features)
train_data = splits['train']
tensor_target = torch.tensor(train_data['out']).float()
error = nn.BCEWithLogitsLoss(pos_weight=(len(tensor_target)/ tensor_target.sum()))
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, amsgrad=True, weight_decay=0.1)

for epoch in range(num_epochs):
    for batch in train_loader:
        
        # Stop if the (last) batch is not big enough
        # if len(batch) != 15:
        #     print(batch)
        #     break
        
        train = batch[:, :-1].reshape((batch_size, seq_length, input_dim))
        targets = batch[:, -1:]

        # Clear gradients
        optimizer.zero_grad()
        
        # Forward propagation
        model.train()
        outputs = model(train)
        
        # Calculate softmax and cross entropy loss
        loss = error(outputs.T, targets)

        # Calculating gradients
        loss.backward()
        
        # Update parameters
        optimizer.step()
        
        count += 1
        
        # Calculate Accuracy
        model.eval()
        if count % 100 == 0:

            correct = 0

            # Iterate through 1000 batches in test dataset
            for _ in range(1000):

                # load a new batch
                batch = next(test_iter)

                # the iter will eventually run out, reload it              
                if True in torch.isnan(batch):
                    test_iter = iter(test_loader)
                    batch = next(test_iter)
            
                # seperate features and target
                test_features = batch[:, :-1].reshape((batch_size, seq_length, input_dim))
                test_targets = batch[:,-1:]
                
                # Forward propagation
                outputs = model(test_features)
                
                # Round predictions to get final prediction
                predicted = torch.round(outputs.T)
                
                # print(predicted, test_targets)
                correct += (predicted == test_targets).sum()
                
            total = batch.shape[0]*1000
            accuracy = (correct / float(total)) * 100
            positives = predicted.sum()
                
            # store loss and iteration
            iteration_list.append(count)
            loss_list.append(loss)
            accuracy_list.append(accuracy)

            # Print results
            print(f'Iteration: {count}  Loss: {loss}  Accuracy: {accuracy} % N_postive: {positives}')

Iteration: 100  Loss: 1.122471570968628  Accuracy: 31.393333435058594 % N_postive: 11.0
Iteration: 200  Loss: 0.9584335088729858  Accuracy: 53.73332977294922 % N_postive: 4.0
Iteration: 300  Loss: 7.438199043273926  Accuracy: 37.16666793823242 % N_postive: 11.0
Iteration: 400  Loss: 0.8191106915473938  Accuracy: 36.833335876464844 % N_postive: 11.0
Iteration: 500  Loss: 10.560690879821777  Accuracy: 98.43333435058594 % N_postive: 0.0
Iteration: 600  Loss: 0.6931473016738892  Accuracy: 99.08000183105469 % N_postive: 0.0
Iteration: 700  Loss: 0.6931473016738892  Accuracy: 98.35333251953125 % N_postive: 0.0


KeyboardInterrupt: 

In [ ]:
""" Model Evaluation """

' Model Evaluation '